# Data Variation (total 3 million)

### Load clean address data with labelled components

In [23]:
import pandas as pd
import numpy as np
import re
import csv
import itertools

In [24]:
df_clean_w_labels = pd.read_pickle("./data/data_creation/cleaned_data_w_labels.pkl")
df_clean_w_labels

,address,block,street,building,floor_num,unit_num,postal_code,country
0,BLOCK 170 ANG MO KIO AVENUE 4 KEBUN BARU LINK ...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,22,49,560170,SINGAPORE
1,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #15-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,15,17,560170,SINGAPORE
2,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #27-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,27,49,560170,SINGAPORE
3,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #15-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,15,42,560170,SINGAPORE
4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #25-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,25,34,560170,SINGAPORE
...,...,...,...,...,...,...,...,...
864478,863 WOODLANDS STREET 83 MY WORLD PRESCHOOL LTD...,863,WOODLANDS STREET 83,MY WORLD PRESCHOOL LTD.,20,80,730863,SINGAPORE
864479,BLOCK 863 WOODLANDS STREET 83 MY WORLD PRESCHO...,863,WOODLANDS STREET 83,MY WORLD PRESCHOOL LTD.,29,97,730863,SINGAPORE
864480,863 WOODLANDS STREET 83 MY WORLD PRESCHOOL LTD...,863,WOODLANDS STREET 83,MY WORLD PRESCHOOL LTD.,17,07,730863,SINGAPORE
864481,864 WOODLANDS STREET 83 #07-57 SINGAPORE 730864,864,WOODLANDS STREET 83,NaN,07,57,730864,SINGAPORE


In [29]:
df2 = pd.read_pickle("./data/data_creation/cleaned_data_w_extended_fields.pkl")
# note: this dataframe contains more columns such as floor_unit and country_postal

In [30]:
data_to_be_varied = df2.sample(frac=1).reset_index(drop=True)
data_to_be_varied

,address,block,street,street_name,street_num,building,floor_unit,floor_num,unit_num,country,postal_code,country_postal
0,374 BUKIT BATOK STREET 31 #15-10 SINGAPORE 650374,374,BUKIT BATOK STREET 31,BUKIT BATOK STREET,31,,#15-10,15,10,SINGAPORE,650374,SINGAPORE 650374
1,115 PENDING ROAD #28-72 SINGAPORE 670115,115,PENDING ROAD,PENDING ROAD,,,#28-72,28,72,SINGAPORE,670115,SINGAPORE 670115
2,230 PASIR RIS STREET 21 #24-19 SINGAPORE 510230,230,PASIR RIS STREET 21,PASIR RIS STREET,21,,#24-19,24,19,SINGAPORE,510230,SINGAPORE 510230
3,126 YISHUN STREET 11 CHONG PANG VIEW #28-39 SI...,126,YISHUN STREET 11,YISHUN STREET,11,CHONG PANG VIEW,#28-39,28,39,SINGAPORE,760126,SINGAPORE 760126
4,988C JURONG WEST STREET 93 #26-70 SINGAPORE 64...,988C,JURONG WEST STREET 93,JURONG WEST STREET,93,,#26-70,26,70,SINGAPORE,643988,SINGAPORE 643988
...,...,...,...,...,...,...,...,...,...,...,...,...
864478,179 BUKIT BATOK WEST AVENUE 8 #21-96 SINGAPORE...,179,BUKIT BATOK WEST AVENUE 8,BUKIT BATOK WEST AVENUE,8,,#21-96,21,96,SINGAPORE,650179,SINGAPORE 650179
864479,334 HOUGANG AVENUE 5 #19-41 SINGAPORE 530334,334,HOUGANG AVENUE 5,HOUGANG AVENUE,5,,#19-41,19,41,SINGAPORE,530334,SINGAPORE 530334
864480,BLOCK 323 SEMBAWANG CLOSE #01-04 SINGAPORE 750323,323,SEMBAWANG CLOSE,SEMBAWANG CLOSE,,,#01-04,01,04,SINGAPORE,750323,SINGAPORE 750323
864481,418 ANG MO KIO AVENUE 10 TECK GHEE HEARTLANDS ...,418,ANG MO KIO AVENUE 10,ANG MO KIO AVENUE,10,TECK GHEE HEARTLANDS,#12-50,12,50,SINGAPORE,560418,SINGAPORE 560418


In [56]:
def permute_address(df, fields_to_permute):
    '''
    Arguments:
        df (DataFrame): source dataframe
        fields_to_permute (List): address components which will be permuted (eg. ['street', 'building'])
    Return:
        df_permuted (DataFrame): dataframe with permuted addresses
    '''
    
    df_permuted = pd.DataFrame(columns=df.columns)

    # generate all permutations of the fields to be permuted
    field_permutations = list(itertools.permutations(fields_to_permute))

    # generate multiple permutations of the address and append to the new DataFrame
    for permutation in field_permutations:
        df_copy = df.copy()

        for index, row in df_copy.iterrows():
            # create a dict for the permuted addresses
            permuted_address = {}
            for i, field in enumerate(permutation):
                value = row[field]
                if pd.isna(value) or value=='':
                    continue
                # add the field-value pair to the permuted address dictionary
                permuted_address[field] = value

            # print(permuted_address.values())
            permuted_mid = ' '.join(permuted_address.values())
            permuted_address_str = str(row['block']) + " " + permuted_mid + " " + str(row['country_postal'])

            # update the 'address' column with the permuted address
            df_copy.at[index, 'address'] = permuted_address_str
            
        df_permuted = pd.concat([df_permuted, df_copy]).drop_duplicates(subset=['address'], ignore_index=True).replace(r'^\s*$', np.nan, regex=True)
    df_permuted = df_permuted[['address', 'block', 'street', 'building', 'floor_num', 'unit_num', 'postal_code', 'country']]

    return df_permuted

### Permute Kaggle data

In [41]:
fields = ['street', 'floor_unit', 'building']

df_permuted_w_labels = permute_address(data_to_be_varied, fields)
df_permuted_w_labels

,address,block,street,building,floor_num,unit_num,postal_code,country
0,374 BUKIT BATOK STREET 31 #15-10 SINGAPORE 650374,374,BUKIT BATOK STREET 31,NaN,15,10,650374,SINGAPORE
1,115 PENDING ROAD #28-72 SINGAPORE 670115,115,PENDING ROAD,NaN,28,72,670115,SINGAPORE
2,230 PASIR RIS STREET 21 #24-19 SINGAPORE 510230,230,PASIR RIS STREET 21,NaN,24,19,510230,SINGAPORE
3,126 YISHUN STREET 11 #28-39 CHONG PANG VIEW SI...,126,YISHUN STREET 11,CHONG PANG VIEW,28,39,760126,SINGAPORE
4,988C JURONG WEST STREET 93 #26-70 SINGAPORE 64...,988C,JURONG WEST STREET 93,NaN,26,70,643988,SINGAPORE
...,...,...,...,...,...,...,...,...
3302432,60 MARINE DRIVE GARDENS #06-27 MARINE DRIVE SI...,60,MARINE DRIVE,MARINE DRIVE GARDENS,06,27,440060,SINGAPORE
3302433,428 GARDEN HILL #09-13 BEDOK NORTH ROAD SINGAP...,428,BEDOK NORTH ROAD,GARDEN HILL,09,13,460428,SINGAPORE
3302434,286 FICUS VALE #12-06 YISHUN AVENUE 6 SINGAPOR...,286,YISHUN AVENUE 6,FICUS VALE,12,06,760286,SINGAPORE
3302435,43 AYER RAJAH NEIGHBOURHOOD POLICE POST #19-17...,43,TEBAN GARDENS ROAD,AYER RAJAH NEIGHBOURHOOD POLICE POST,19,17,600043,SINGAPORE


In [42]:
df_permuted_w_labels.to_pickle("./data/data_variation/permuted_data_w_labels.pkl")
df_permuted_w_labels.to_csv(r'./data/data_variation/permuted_data_w_labels.csv', index=False)

### Permute webscraped data

In [52]:
df_webscraped_w_labels = pd.read_csv("./data/webscraped_data/webscraped_data_w_labels.csv", dtype=str).dropna(subset=['address',]).drop_duplicates(subset=['address'], ignore_index=True).replace(r'^\s*$', np.nan, regex=True)
df_webscraped_w_labels

,address,block,street,building,floor_num,unit_num,country,postal_code
0,21 THIRD CHIN BEE ROAD SINGAPORE 618694,21,THIRD CHIN BEE ROAD,NaN,NaN,NaN,SINGAPORE,618694
1,6 UBI ROAD 1 #04-09 WINTECH CENTRE SINGAPORE 4...,6,UBI ROAD 1,WINTECH CENTRE,4,9,SINGAPORE,408726
2,52 UBI AVENUE 3 #04-39/40 FRONTIER SINGAPORE 4...,52,UBI AVENUE 3,FRONTIER,4,39/40,SINGAPORE,408867
3,1 KAKI BUKIT ROAD 1 #03-17 ENTERPRISE ONE SING...,1,KAKI BUKIT ROAD 1,ENTERPRISE ONE,3,17,SINGAPORE,415934
4,8 TUAS VIEW CIRCUIT SINGAPORE 637777,8,TUAS VIEW CIRCUIT,NaN,NaN,NaN,SINGAPORE,637777
...,...,...,...,...,...,...,...,...
5896,3017 BEDOK NORTH STREET 5 #06-19 GOURMET EAST ...,3017,BEDOK NORTH STREET 5,GOURMET EAST KITCHEN,6,19,SINGAPORE,486121
5897,BLOCK 15 WOODLANDS LOOP #01-01 WOODLANDS EAST ...,15,WOODLANDS LOOP,WOODLANDS EAST INDUSTRIAL PARK,1,1,SINGAPORE,738322
5898,15 WOODLANDS LOOP #03-00A WOODLANDS EAST INDUS...,15,WOODLANDS LOOP,WOODLANDS EAST INDUSTRIAL PARK,3,00A,SINGAPORE,738322
5899,10 UBI CRESCENT #05-92 TO 95 UBI TECHPARK (LOB...,10,UBI CRESCENT,TO 95 UBI TECHPARK (LOBBY E),5,92,SINGAPORE,408564


In [61]:
floor_unit_list = []
country_postal_list = []

for index, row in df_webscraped_w_labels.iterrows():
    if pd.isna(row["floor_num"]) or row["floor_num"]=='' or pd.isna(row["unit_num"]) or row["unit_num"]=='':
        floor_unit = ''
    else:
        floor_unit = '#' + str(row['floor_num']) + '-' + str(row['unit_num'])
    
    if pd.isna(row["country"]) or row["country"]=='' or pd.isna(row["postal_code"]) or row["postal_code"]=='':
        country_postal = ''
    else:
        country_postal = str(row['country']) + ' ' + str(row['postal_code'])
        
    floor_unit_list.append(floor_unit)
    country_postal_list.append(country_postal)
    
df_webscraped_w_labels = df_webscraped_w_labels.assign(floor_unit = floor_unit_list, country_postal = country_postal_list)    
    
fields = ['street', 'floor_unit', 'building']

df_web_permuted = permute_address(df_webscraped_w_labels, fields)
df_web_permuted

,address,block,street,building,floor_num,unit_num,postal_code,country
0,21 THIRD CHIN BEE ROAD SINGAPORE 618694,21,THIRD CHIN BEE ROAD,NaN,NaN,NaN,618694,SINGAPORE
1,6 UBI ROAD 1 #4-9 WINTECH CENTRE SINGAPORE 408726,6,UBI ROAD 1,WINTECH CENTRE,4,9,408726,SINGAPORE
2,52 UBI AVENUE 3 #4-39/40 FRONTIER SINGAPORE 40...,52,UBI AVENUE 3,FRONTIER,4,39/40,408867,SINGAPORE
3,1 KAKI BUKIT ROAD 1 #3-17 ENTERPRISE ONE SINGA...,1,KAKI BUKIT ROAD 1,ENTERPRISE ONE,3,17,415934,SINGAPORE
4,8 TUAS VIEW CIRCUIT SINGAPORE 637777,8,TUAS VIEW CIRCUIT,NaN,NaN,NaN,637777,SINGAPORE
...,...,...,...,...,...,...,...,...
20970,896 SIME DARBY CENTRE #4-05E DUNEARN ROAD SING...,896,DUNEARN ROAD,SIME DARBY CENTRE,4,05E,589472,SINGAPORE
20971,3017 GOURMET EAST KITCHEN #6-19 BEDOK NORTH ST...,3017,BEDOK NORTH STREET 5,GOURMET EAST KITCHEN,6,19,486121,SINGAPORE
20972,15 WOODLANDS EAST INDUSTRIAL PARK #1-1 WOODLAN...,15,WOODLANDS LOOP,WOODLANDS EAST INDUSTRIAL PARK,1,1,738322,SINGAPORE
20973,15 WOODLANDS EAST INDUSTRIAL PARK #3-00A WOODL...,15,WOODLANDS LOOP,WOODLANDS EAST INDUSTRIAL PARK,3,00A,738322,SINGAPORE


### Combine clean data and permuted data

In [62]:
df_combined = pd.concat([df_clean_w_labels, df_permuted_w_labels, df_webscraped_w_labels, df_web_permuted]).drop_duplicates(subset=['address'], ignore_index=True)[['address', 'block', 'street', 'building', 'floor_num', 'unit_num', 'postal_code', 'country']]
df_combined

,address,block,street,building,floor_num,unit_num,postal_code,country
0,BLOCK 170 ANG MO KIO AVENUE 4 KEBUN BARU LINK ...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,22,49,560170,SINGAPORE
1,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #15-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,15,17,560170,SINGAPORE
2,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #27-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,27,49,560170,SINGAPORE
3,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #15-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,15,42,560170,SINGAPORE
4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 #25-...,170,ANG MO KIO AVENUE 4,KEBUN BARU LINK 1,25,34,560170,SINGAPORE
...,...,...,...,...,...,...,...,...
3449103,896 SIME DARBY CENTRE #4-05E DUNEARN ROAD SING...,896,DUNEARN ROAD,SIME DARBY CENTRE,4,05E,589472,SINGAPORE
3449104,3017 GOURMET EAST KITCHEN #6-19 BEDOK NORTH ST...,3017,BEDOK NORTH STREET 5,GOURMET EAST KITCHEN,6,19,486121,SINGAPORE
3449105,15 WOODLANDS EAST INDUSTRIAL PARK #1-1 WOODLAN...,15,WOODLANDS LOOP,WOODLANDS EAST INDUSTRIAL PARK,1,1,738322,SINGAPORE
3449106,15 WOODLANDS EAST INDUSTRIAL PARK #3-00A WOODL...,15,WOODLANDS LOOP,WOODLANDS EAST INDUSTRIAL PARK,3,00A,738322,SINGAPORE


In [63]:
df_combined.to_pickle("./data/data_variation/combined_data_w_labels.pkl")
df_combined.to_csv(r'./data/data_variation/combined_data_w_labels.csv', index=False)